# 線形システムにおけるSystem Level Synthesis (SLS)

参考：
* [Lecture 11: System Level Synthesis and Robust Control Bounds](https://nikolaimatni.github.io/courses/ese680-fall2019/scribe-notes/lecture11.pdf)：これが一番わかり易いかも
* [System Level Synthesis](https://arxiv.org/abs/1904.01634)：元論文
* [Reinforcement Learning: Theory and Algorithms](https://rltheorybook.github.io/)
* [On the Sample Complexity of the Linear Quadratic Regulator](https://arxiv.org/abs/1710.01688)のAppendix C．最適化は5.2章を参照
* [Safely Learning to Control the Constrained Linear Quadratic Regulator](https://arxiv.org/abs/1809.10121)

今回はSystem Level Synthesis (SLS) について学びます．

[RL_LQR.ipynb](RL_LQR.ipynb)では二次の目的関数と次の線形なシステムを考えました．

$$
\begin{aligned}
&\lim _{T \rightarrow \infty} \frac{1}{T} \sum_{t=1}^T \mathbb{E}\left[x_t^* Q x_t+u_t^* R u_t\right]\\
&x_{t+1}=A x_t+B u_t+w_t, \quad x_0 \sim \mathcal{D}, w_t \sim \mathcal{N}\left(0, \sigma_w^2 I\right)
\end{aligned}
$$

ここで$x_t \in \mathbb{R}^n$，$u_t \in \mathbb{R}^p$，$w_t \in \mathbb{R}^n$とします．

SLSを使うと嬉しいことがいくつかあります．まず，この問題は$u$について凸ではありませんが，SLSを使うと凸最適化問題に変形することができます．
また，SLSを使うとロバストなLQRや制約付きLQRが簡単に実現できます．

## 有限ホライゾン

次の時間非定常システムと非定常コントローラーを考えます．
$$
x_{t+1}=A_t x_t+B_t u_t+w_t
, \quad
u_t=\sum_{\tau=0}^t K^{t, t-\tau} x_\tau
$$

このような状態と制御入力のベクトルはノイズベクトルを使って書き表せます．

---

**ノイズからシステム応答への変換**

また，システムの要素を次のようにベクトルと行列形式で表します．

$$
\mathbf{x}=\left[\begin{array}{c}
x_0 \\
x_1 \\
\vdots \\
x_T
\end{array}\right], \mathbf{u}=\left[\begin{array}{c}
u_0 \\
u_1 \\
\vdots \\
u_T
\end{array}\right] \mathbf{w}=\left[\begin{array}{c}
x_0 \\
w_0 \\
w_1 \\
\vdots \\
w_{T-1}
\end{array}\right], \mathbf{K}=\left[\begin{array}{cccc}
K^{0,0} & & & \\
K^{1,1} & K^{1,0} & & \\
\vdots & \ddots & \ddots & \\
K^{T, T} & \ldots & K^{T, 1} & K^{T, 0}
\end{array}\right]
$$

また，$Z$を次のようなdown-shift作用素とします（劣対角成分が単位行列）．

$$
Z=\left[\begin{array}{cccc}
0 & & & \\
I & 0 & & \\
\vdots & \ddots & \ddots & \\
0 & \ldots & I& 0
\end{array}\right]
$$

さらに，

$$
\mathcal{A}=\left[\begin{array}{ccccc}
A_0 & & & &\\
& A_1 & & &\\
& & \ddots & &\\
& & & A_{T-1} &\\
& & & & 0
\end{array}\right]\quad
\mathcal{B}=\left[\begin{array}{ccccc}
B_0 & & & &\\
& B_1 & & &\\
& & \ddots & &\\
& & & B_{T-1} &\\
& & & & 0
\end{array}\right]
$$

とします．このとき，
$$
\begin{aligned}
x_{t+1}
&=A_t x_t+B_t u_t+w_t\\
&=A_t x_t+B_t \sum_{\tau=0}^t K^{t, t-\tau} x_\tau + w_t\\
&=A_t x_t+B_t \left(K^{0, 0} x_0 + K^{1, 1}x_0 + K^{1, 0}x_1 + \cdots\right) + w_t
\end{aligned}
$$

なので，これは$Z$を使って行列形式で書くことができます：

$$
\begin{aligned}
&\mathbf{x}-Z(\mathcal{A}+\mathcal{B} \mathbf{K})\mathbf{x}=\mathbf{w}\\
\Leftrightarrow&\left(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K})\right)\mathbf{x}=\mathbf{w}\\
\Leftrightarrow&\mathbf{x}=\left(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K})\right)^{-1}\mathbf{w}\\
\end{aligned}
$$

ここで，$\mathcal{A}$と$\mathcal{B}$が対角行列かつ$K$が下三角行列であり，$I$が単位行列なため，
$\left(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K})\right)$に逆行列が存在することを使ってます．

まとめると，

$$
\begin{aligned}
& \mathbf{x}=(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K}))^{-1} \mathbf{w} \\
& \mathbf{u}=\mathbf{K}(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K}))^{-1} \mathbf{w}
\end{aligned}
$$

と簡単に書けます．これはさらに

$$
\boldsymbol{\Phi}_x=\left[\begin{array}{cccc}
\Phi_x^{0,0} & & & \\
\Phi_x^{1,1} & \Phi_x^{1,0} & & \\
\vdots & \ddots & \ddots & \\
\Phi_x^{T, T} & \cdots & \Phi_x^{T, 1} & \Phi_x^{T, 0}
\end{array}\right], \boldsymbol{\Phi}_u=\left[\begin{array}{cccc}
\Phi_u^{0,0} & & & \\
\Phi_u^{1,1} & \Phi_u^{1,0} & & \\
\vdots & \ddots & \ddots & \\
\Phi_u^{T, T} & \cdots & \Phi_u^{T, 1} & \Phi_u^{T, 0}
\end{array}\right]
$$

なる下三角ブロック行列を使えば，

$$
\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]=\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \mathbf{w}
$$

として簡単に書けます．

---


ここで，次が成立します．

---

下三角行列$\mathbf{K}$とコントローラー$\mathbf{u}=\mathbf{K} \mathbf{x}$を考えます．このとき，次の両方が成立します：

1.

$$
\left[\begin{array}{ll}
I-Z \mathcal{A}, & -Z \mathcal{B}
\end{array}\right]\left[\begin{array}{l}
\mathbf{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I
$$

を満たす下三角ブロック行列$\left(\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u\right)$について，
$
\mathcal{S}=\left\{
(\mathbf{x}, \mathbf{u})
\vert
\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]
=\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \mathbf{w}
\right\}
$
とする．
また，$\mathcal{S}'$をあり得る全てのシステム応答の集合とすると，
$\mathcal{S}' \subset \mathcal{S}$．



2.

上記の制約を満たす$\left(\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u\right)$について，$\mathbf{K}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}$が
$
\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]=\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \mathbf{w}
$
を与える．


**$1$の証明**

まず，明らかに
$$
[I-Z \mathcal{A},\; -Z \mathcal{B}]\left[\begin{array}{c}
(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K}))^{-1} \\
\mathbf{K}(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K}))^{-1}
\end{array}\right]=(I-Z \mathcal{A}-Z \mathcal{B} \mathbf{K})(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K}))^{-1}=I
$$

です．一方で，上で見たように
$$
\begin{aligned}
& \mathbf{x}=(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K}))^{-1} \mathbf{w} \\
& \mathbf{u}=\mathbf{K}(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K}))^{-1} \mathbf{w}
\end{aligned}
$$
なので，１が成立します．

**$2$の証明**

まず，制約より$\boldsymbol{\Phi}_x$の対角ブロックが$I$であることから，
$\boldsymbol{\Phi}_x^{-1}$
が存在することがわかります．
また，
$\mathbf{x}=(I-Z(\mathcal{A}+\mathcal{B} \mathbf{K}))^{-1} \mathbf{w}$に$\mathbf{K}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}$を代入して，
$\mathbf{x}=\left(I-Z\left(\mathcal{A}+\mathcal{B} \boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}\right)\right)^{-1} \mathbf{w}$
です．

ここで，
$$
\left(I-Z\left(\mathcal{A}+\mathcal{B} \boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}\right)\right)^{-1}=\left(\left((I-Z \mathcal{A}) \boldsymbol{\Phi}_x-Z \mathcal{B} \boldsymbol{\Phi}_u\right) \boldsymbol{\Phi}_x^{-1}\right)^{-1}=\boldsymbol{\Phi}_x\left((I-Z \mathcal{A}) \boldsymbol{\Phi}_{\mathbf{x}}-Z \mathcal{B} \boldsymbol{\Phi}_u\right)^{-1}=\boldsymbol{\Phi}_x
$$
が成立します（最後の等式は１の制約のせいです）．
よって
$\mathbf{x}=\Phi_x \mathbf{x}$です．

さらに，
$\mathbf{u}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1} \mathbf{x}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1} \boldsymbol{\Phi}_x \mathbf{w}=\boldsymbol{\Phi}_u \mathbf{w}$
なので，

$\left(\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u\right)$が制約を満たしているなら，
$\mathbf{K}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}$が
$
\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]=\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \mathbf{w}
$
を与えます．

---




上ではSLSを使ってシステムを変形できることを見ました．
続いて最適な制御器を計算する方法を見ていきます．

### ノイズなしのLQR

次のLQR問題を考えてみます．

$$
\begin{array}{cl}
\min _{x_t, u_t} & \sum_{t=0}^T x_t^{\top} Q_t x_t+u_t^{\top} R_t u_t \\
\text { subject to } & x_{t+1}=A x_t+B u_t, t=0, \ldots, T-1 \\
& x_0 \text { known } \\
\end{array}
$$

これはベクトル形式で表せば，次と等価です．

$$
\begin{aligned}
\min _{\mathbf{x}, \mathbf{u}}  & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]\right\|_F^2 \\
\text { subject to } \mathbf{x} & =Z \mathcal{A} \mathbf{x}+Z \mathcal{B} \mathbf{u} 
\end{aligned}
$$

ここで，$\mathcal{Q}:=\operatorname{blkdiag}\left(Q_0, Q_1, \ldots, Q_T\right), \mathcal{R}:=\operatorname{blkdiag}\left(R_0, R_1, \ldots, R_T\right)$
としました．

続いて，
$\mathbf{w}=\left[\begin{array}{c}x_0 \\ 0 \\ \vdots \\ 0\end{array}\right]$
および
$\boldsymbol{\Phi}(:, 0)$
を$\mathbf{\Phi}$の最初のブロック列とすれば，

$$
\mathbf{x}=\boldsymbol{\Phi}_x \mathbf{w}=\boldsymbol{\Phi}_x(:, 0) x_0 \text { and } \mathbf{u}=\boldsymbol{\Phi}_u \mathbf{w}=\boldsymbol{\Phi}_u(:, 0) x_0
$$

であることがわかります．
よって，SLSを使えば，ノイズなしLQRは

$$
\begin{gathered}
\min _{\Phi_x, \Phi_u}\left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{c}
\mathbf{\Phi}_x(:, 0) \\
\mathbf{\Phi}_u(:, 0)
\end{array}\right] x_0\right\|_F^2 \\
\text { subject to }[I-Z \mathcal{A}, -Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x(:, 0) \\
\mathbf{\Phi}_u(:, 0)
\end{array}\right]=I 
\end{gathered}
$$

で解くことができます．


### ノイズあり＆$x_0$がランダムなとき

$\mathbf{w} \stackrel{\text { i.i.d. }}{\sim} \mathcal{N}\left(0, \Sigma_w\right)$の場合を考えてみましょう．つまり，

$$
\begin{aligned}
\min _{x_t, u_t} & \sum_{t=0}^T \mathbb{E}\left[x_t^{\top} Q_t x_t+u_t^{\top} R_t u_t\right] \\
\text { subject to } & x_{t+1}=A x_t+B u_t+w_t .
\end{aligned}
$$

を解きます．これはSLSに直せば

$$
\begin{aligned}
& \min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u}\left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \Sigma_w^{1 / 2}\right\|_F^2 \\
& \text { subject to }\left[\begin{array}{ll}
I-Z \mathcal{A} & -Z \mathcal{B}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]=I \text {, } \\
&
\end{aligned}
$$

と同じです．

特に$\Sigma_w=I$ならばノイズなしのときと同じです（[RL_LQR_as_SDP.ipynb](RL_LQR_as_SDP.ipynb)でやりましたね）．


### Pythonで実験


In [16]:
def assert_AB_consistent(A, B):
    assert len(A.shape) == 2 and A.shape[0] == A.shape[1]
    assert len(B.shape) == 2
    assert A.shape[0] == B.shape[0]


def assert_ABQR_consistent(A, B, Q, R):
    assert_AB_consistent(A, B)

    assert len(Q.shape) == 2
    assert len(R.shape) == 2

    assert Q.shape[1] == A.shape[0]
    assert R.shape[1] == B.shape[1]


def assert_ABCD_consistent(A, B, C, D):
    assert_AB_consistent(A, B)

    assert len(C.shape) == 2
    assert len(D.shape) == 2

    assert C.shape[1] == A.shape[0]
    assert C.shape[0] == D.shape[0]
    assert D.shape[1] == B.shape[1]

In [17]:
# 今回は面倒なので定常システムを扱います．

import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera


def inverted_pendulum_dynamics(dt: float = 1.0):
    ''' 倒立振子のダイナミクスを返します．
    参考（式20）：https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SystemModeling
    状態：[x, x', phi, phi']
    '''
    M = 0.5  # mass of the cart
    m = 0.2  # mass of the pendulum

    I = 0.006   # mass moment of inertia
    l = 0.3  # length to pendulum center of mass

    b = 0.1  # coefficient of friction for cart
    g = 9.8   # gravity

    X = I * (M + m) + M * m * (l ** 2)
    A = np.array([
        [0, 1, 0, 0],
        [0, -(I + m*(l**2))*b / X, (m**2) * g * (l**2) / X, 0],
        [0, 0, 0, 1],
        [0, -m*l*b / X, m*g*l*(M+m) / X, 0] 
    ]) * dt   

    B = np.array([0, (I + m*(l**2)) / X, 0, m*l / X]).reshape((4, 1))

    return A, B


def draw_inverted_pendulum(camera: Camera, states_list: np.ndarray, title_list: str):
    '''倒立振子を描画します
    Args:
        states_list ([np.ndarray, np.ndarray, ...]): horizon x (state dim)
        title_list ([str, str, ...])
    '''

    num_states = len(states_list)

    def _draw(x):
        cart_pos = x[0]
        l = 0.3
        pendulum_x = cart_pos + 1 * np.sin(x[2])
        pendulum_y = 1 * np.cos(x[2])

        plt.hlines(0, -2, 2, color="black")
        plt.plot([cart_pos, pendulum_x], [0, pendulum_y], color="black")

    states_list = np.array(states_list)
    T = states_list.shape[1]

    for t in range(T):
        for i in range(num_states):
            plt.subplot(1, num_states, i+1)
            _draw(states_list[i, t])
            plt.title(title_list[i])
            plt.xlim([-2, 2])
            plt.ylim([-2, 2])
            plt.xlabel("x [m]")
            plt.ylabel("y [m]")

        camera.snap()


def cost_LQR_samples(states, inputs, Q, R):
    """
    LQRの期待コストをサンプルから計算します．
    """
    assert states.shape[1] == Q.shape[0]
    assert inputs.shape[1] == R.shape[0]
    horizon = states.shape[0]

    cost = np.trace(states @ Q @ states.T) + np.trace(inputs @ R @ inputs.T)
    return cost / horizon


まず通常の制約なしSLSを解くことを考えてみます．
上で見たように，

$$
\begin{gathered}
\min _{\Phi_x, \Phi_u}\left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{c}
\mathbf{\Phi}_x(:, 0) \\
\mathbf{\Phi}_u(:, 0)
\end{array}\right] x_0\right\|_F^2 \\
\text { subject to }[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x(:, 0) \\
\mathbf{\Phi}_u(:, 0)
\end{array}\right]=I
\end{gathered}
$$

こうやっても解けますが，後のために

$$
\begin{gathered}
\min _{\Phi_x, \Phi_u}\left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{c}
\mathbf{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right] \mathbf{w}\right\|_F^2 \\
\text { subject to }[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I
\end{gathered}
$$

を解いてみます．


In [18]:
from scipy.linalg import block_diag


def make_block_AB(T: int, A, B):
    zero_A = np.zeros_like(A)
    A_block = block_diag(*[A]*T, zero_A)
    zero_B = np.zeros_like(B)
    B_block = block_diag(*[B]*T, zero_B)
    n_block = A.shape[0] * (T + 1)
    assert A_block.shape[0] == A_block.shape[1] == n_block

    p_block = B.shape[1] * (T + 1)
    assert B_block.shape[0] == A_block.shape[0]
    assert B_block.shape[1] == p_block

    return A_block, B_block


def make_block_QR(T: int, Q, R):
    Q_block = block_diag(*[Q]*(T+1))
    R_block = block_diag(*[R]*(T+1))
    n_block = Q.shape[0] * (T + 1)
    assert Q_block.shape[0] == n_block

    p_block = R.shape[0] * (T + 1)
    assert R_block.shape[0] == p_block

    return Q_block, R_block


def make_down_shift_Z(T: int, x_dim: int):
    n = x_dim
    I = np.eye(n)

    Z = block_diag(*[I]*T)

    Z = np.vstack([np.zeros((n, n*T)), Z])
    Z = np.hstack([Z, np.zeros((n*(T+1), n))])
    return Z


In [19]:
import cvxpy as cp


def psd_sqrt(P):
    '''
    行列Pの平方根を計算します．
    https://kagamiz.hatenablog.com/entry/2020/05/16/212214 参照．
    '''
    assert len(P.shape) == 2
    assert P.shape[0] == P.shape[1]
    w, v = np.linalg.eigh(P)
    assert (w >= 0).all()
    return v @ np.diag(np.sqrt(w)) @ v.T


def solve_LQR_no_noise(A, B, Q, R, T, x0):
    """与えられたx0について，ノイズなしのSLSを解きます．"""

    assert_ABQR_consistent(A, B, Q, R)
    assert T >= 1
    n, p = B.shape

    # コストを定義します
    Q_sqrt = psd_sqrt(Q)
    R_sqrt = psd_sqrt(R)

    Z = make_down_shift_Z(T, n)
    A_block, B_block = make_block_AB(T, A, B)
    Q_block, R_block = make_block_QR(T, Q_sqrt, R_sqrt)
    QR_block = block_diag(Q_block, R_block)

    Phi = cp.Variable(((T+1)*(n+p), (T+1)*n))

    w = np.zeros((T+1)*n)
    w[:n] = x0

    cost = cp.norm(QR_block @ Phi @ w, p="fro")

    # 制約を定義します
    I = np.eye(Z.shape[0])
    const_block = np.block([I - Z @ A_block, - Z @ B_block])
    const = [const_block @ Phi == I]

    prob = cp.Problem(cp.Minimize(cost), const)
    prob.solve()

    if prob.status == cp.OPTIMAL:
        Phi = Phi.value
        Phi_x = Phi[:(T+1)*n]
        Phi_u = Phi[(T+1)*n:]
        K = Phi_u @ np.linalg.inv(Phi_x)
        return True, K, prob.value
    else:
        return False, None, None

In [20]:
def sls_roll_forward(A, B, K, x0, T, sigma_w=0):
    """SLSで計算したKでシステムを走らせます．
    """
    assert_AB_consistent(A, B)

    states = []
    inputs = []
    x = x0.copy()
    xs = [x]
    n = A.shape[0]

    for t in range(T):
        k = K[t]
        u = np.zeros(B.shape[1])
        for h, y in enumerate(xs[::-1]):  # xは逆から見ていきます
            u = u + k[h*n:(h+1)*n] @ y
        states.append(x)
        inputs.append(u)
        noise = np.random.normal(x.shape) * sigma_w
        x = A @ x + B @ u + noise

    states.append(x)
    inputs.append(u)
    return np.array(states), np.array(inputs)


In [22]:
A, B = inverted_pendulum_dynamics(0.1)
Q = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 0],
])
R = np.eye(1)
T = 30

x0 = np.array([0.1, -0.2, 0.1, -0.1])

_, K, cost = solve_LQR_no_noise(A, B, Q, R, T, x0)

states, inputs = sls_roll_forward(A, B, K, x0, T)

print("LQRのコスト", cost)
print("Rolloutしたコスト", cost_LQR_samples(states, inputs, Q, R))

LQRのコスト 0.1463656745860303
Rolloutしたコスト 0.0011462474142719418


In [73]:
fig = plt.figure(figsize=(5, 4))
camera = Camera(fig)
draw_inverted_pendulum(camera, [states], ["LQR"])
animation = camera.animate()
animation.save("./figs/inv_pend_sls.gif")
plt.close()

![sls](./figs/inv_pend_sls.gif)

### $\mathcal{H}_\infty$制御

続いて，次の$\mathcal{H}_\infty$最適制御を扱ってみましょう．

$$\begin{aligned} \min _{\mathbf{x}, \mathbf{u}} \max _{\|\mathbf{w}\|_2 \leq 1} & \sum_{t=0}^T x_t^{\top} Q_t x_t+u_t^{\top} R_t u_t \\ \text { subject to } & x_{t+1}=A x_t+B u_t+w_t .\end{aligned}$$

これをベクトル形式に直すと

$$
\begin{aligned}
\min _{\mathbf{x}, \mathbf{u}} \max _{\|\mathbf{w}\|_2 \leq 1} & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]\right\|_2^2 \\
\text { subject to } & \mathbf{x}=Z \mathcal{A} \mathbf{x}+Z \mathcal{B} \mathbf{u}+\mathbf{w} .
\end{aligned}
$$

であり，更にSLSを使うと

$$
\begin{aligned}
\min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u} \max _{\|\mathbf{w}\|_2 \leq 1} & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \mathbf{w}\right\|_2^2 \\
\text { subject to } & {[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I . }
\end{aligned}
$$

として書けます．ここで，

$$
\max _{\|\mathbf{w}\|_2 \leq 1} \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \mathbf{w}\right\|_2^2
$$

は行列における[作用素ノルム](https://ja.wikipedia.org/wiki/%E8%A1%8C%E5%88%97%E3%83%8E%E3%83%AB%E3%83%A0)です．
特にこれは$2\to 2$のノルムなので，スペクトルノルムに相当します．

スペクトルノルムの最小化はsemidefinite programmingで解けます．

参考：
* [Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf)の4.6.3章

行列$A$のスペクトルノルムは
$$
\|A\|_{n\to m} =\max _{\substack{
\|x\|_{\mathbb{K}^n} \leq 1}}\|A x\|_{\mathbb{K}^m} 
=\max _{\substack{
\|x\|_{\mathbb{K}^n}=1}}\|A x\|_{\mathbb{K}^m}
$$

で定義され，特にスペクトルノルムは$\|A\|_{2\to 2}$です．
ここで，

$$
\|A\|_2 \leq s \text { if and only if } A^T A \preceq s^2 I \text { (and } s \geq 0 \text { ) }
$$

を使えば，スペクトルノルムの最小化は
$
A(\boldsymbol{\Phi}) = 
\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] 
$
として，

$$
\begin{aligned}
\min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u, s} & s
\\
\text { subject to } & A(\boldsymbol{\Phi})^T A(\boldsymbol{\Phi}) \preceq s I
\\
&{[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I . }
\end{aligned}
$$

さらにこれは

$$
\left.A^T A \preceq t^2 I \text { (and } t \geq 0\right) \Longleftrightarrow\left[\begin{array}{cc}
t I & A \\
A^T & t I
\end{array}\right] \succeq 0 \text {. }
$$

を使えば

$$
\begin{aligned}
\min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u, t} & t
\\
\text { subject to } & 
\left[\begin{array}{cc}
t I & A(\boldsymbol{\Phi}) \\
A(\boldsymbol{\Phi})^T & t I
\end{array}\right] \succeq 0
\\
&{[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I . }
\end{aligned}
$$

を解けば良いことがわかります．


In [23]:
def solve_LQR_H_infty(A, B, Q, R, T):
    """与えられたx0について，H∞制御をSLSで解きます．"""

    assert_ABQR_consistent(A, B, Q, R)
    assert T >= 1
    n, p = B.shape
    Phi = cp.Variable(((T+1)*(n+p), (T+1)*n))
    Z = make_down_shift_Z(T, n)
    Q_sqrt = psd_sqrt(Q)
    R_sqrt = psd_sqrt(R)
    A_block, B_block = make_block_AB(T, A, B)
    Q_block, R_block = make_block_QR(T, Q_sqrt, R_sqrt)
    QR_block = block_diag(Q_block, R_block)

    # コストを定義します
    cost = cp.Variable()

    # Phi用の制約を定義します
    I = np.eye(Z.shape[0])
    const_block = np.block([I - Z @ A_block, - Z @ B_block])
    const = [const_block @ Phi == I]

    # コスト用の制約を定義します
    A_Phi = QR_block @ Phi
    tI1 = cost * np.eye(A_Phi.shape[0])
    tI2 = cost * np.eye(A_Phi.shape[1])
    cost_const_mat = cp.bmat([[tI1, A_Phi], [A_Phi.T, tI2]])

    const = [const_block @ Phi == I, cost_const_mat >> 0, cost >= 0]

    prob = cp.Problem(cp.Minimize(cost), const)
    prob.solve()

    if prob.status == cp.OPTIMAL:
        Phi = Phi.value
        Phi_x = Phi[:(T+1)*n]
        Phi_u = Phi[(T+1)*n:]
        K = Phi_u @ np.linalg.inv(Phi_x)
        return True, K, prob.value
    else:
        return False, None, None

In [24]:
T = 10

_, K_Hinfty, cost = solve_LQR_H_infty(A, B, Q, R, T)

states, inputs = sls_roll_forward(A, B, K, x0, T, 0.1)
H_states, H_inputs = sls_roll_forward(A, B, K_Hinfty, x0, T, 0.1)

print("LQRのコスト", cost)
print("通常のLQRがRolloutしたコスト", cost_LQR_samples(states, inputs, Q, R))
print("H∞LQRがRolloutしたコスト", cost_LQR_samples(H_states, H_inputs, Q, R))

LQRのコスト 1.2067593845749707
通常のLQRがRolloutしたコスト 0.5091905410196792
H∞LQRがRolloutしたコスト 0.6340198923621218


In [76]:
fig = plt.figure(figsize=(5*2, 4))
camera = Camera(fig)
draw_inverted_pendulum(camera, [states, H_states], ["LQR", "H∞ LQR"])
animation = camera.animate()
animation.save("./figs/inv_pend_sls_Hinfty.gif")
plt.close()

![sls](./figs/inv_pend_sls_Hinfty.gif)

## SLSによるロバスト制御

SLSを使うとダイナミクスの誤差に対してロバストな制御が達成できます．
見てみましょう．

$$
\boldsymbol{\Delta}=\left[\begin{array}{cccc}
\Delta^{0,0} & & & \\
\Delta^{1,1} & \Delta^{1,0} & & \\
\vdots & \ddots & \ddots & \\
\Delta^{T, T} & \cdots & \Delta^{T, 1} & \Delta^{T, 0}
\end{array}\right]
$$

を任意のブロック下三角行列として，次が成立します．

---

**定理**

次が成立すると仮定する．

$$
\left[\begin{array}{ll}
I-Z \mathcal{A} & -Z \mathcal{B}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]=I+\Delta .
$$


このとき，$\left(I+\Delta^{i, 0}\right)^{-1}$が存在するなら，コントローラー$\mathbf{K}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}$によるシステム応答は

$$
\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]=\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right](I+\boldsymbol{\Delta})^{-1} \mathbf{w}
$$

になる．

**証明**

まず，$\left(I+\Delta^{i, 0}\right)^{-1}$が存在する場合，$(I+\boldsymbol{\Delta})^{-1}$も存在します．このとき，制約は

$$
\left[\begin{array}{ll}
I-Z \mathcal{A} & -Z \mathcal{B}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right](I+\boldsymbol{\Delta})^{-1}=I .
$$

と等価です．よって，コントローラは

$$
\mathbf{K}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}=\boldsymbol{\Phi}_u(I+\boldsymbol{\Delta})^{-1}\left(\boldsymbol{\Phi}_x(I+\boldsymbol{\Delta})^{-1}\right)^{-1}
$$

なので，上の方で見たSLSのシステム応答と照らし合わせれば，定理が成立することがすぐにわかります．

---

この結果を使ってロバストな制御を実現します．
まず，真のシステム行列はわからないですが，$(\widehat{\mathcal{A}}, \widehat{\mathcal{B}})$がわかっている状況を考えましょう．
真のシステムとの差は$\mathcal{D}_{\mathcal{A}}:=\widehat{\mathcal{A}}-\mathcal{A}$ and $\mathcal{D}_{\mathcal{B}}:=\widehat{\mathcal{B}}-\mathcal{B}$とします．

また，$\hat{\boldsymbol{\Phi}}_x$と$\hat{\boldsymbol{\Phi}}_u$は
$$
\left[\begin{array}{ll}
I-Z \hat{\mathcal{A}} & -Z \hat{\mathcal{B}}
\end{array}\right]\left[\begin{array}{l}
\hat{\boldsymbol{\Phi}}_x \\
\hat{\boldsymbol{\Phi}}_u
\end{array}\right]=I
$$
を満たす行列とします．

このとき，
$$
\left[\begin{array}{ll}
I-Z \mathcal{A} & -Z \mathcal{B}
\end{array}\right]\left[\begin{array}{l}
\hat{\mathbf{\Phi}}_x \\
\hat{\mathbf{\Phi}}_u
\end{array}\right]=I+Z\left[\begin{array}{ll}
\mathcal{D}_{\mathcal{A}} & \mathcal{D}_{\mathcal{B}}
\end{array}\right]\left[\begin{array}{l}
\hat{\mathbf{\Phi}}_x \\
\hat{\mathbf{\Phi}}_u
\end{array}\right]
$$
であることがすぐにわかります．すると，上の定理より，$\boldsymbol{\Delta}:=Z\left[\begin{array}{ll}\mathcal{D}_{\mathcal{A}} & \mathcal{D}_{\mathcal{B}}\end{array}\right]\left[\begin{array}{l}\hat{\boldsymbol{\Phi}}_x \\ \hat{\boldsymbol{\Phi}}_u\end{array}\right]$とすれば，コントローラ$\hat{\mathbf{K}}=\hat{\boldsymbol{\Phi}}_u \hat{\boldsymbol{\Phi}}_x^{-1}$は真のシステムにおいて

$$
\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]=\left[\begin{array}{l}
\hat{\boldsymbol{\Phi}}_x \\
\hat{\mathbf{\Phi}}_u
\end{array}\right](I+\boldsymbol{\Delta})^{-1} \mathbf{w} .
$$

の応答をすることがわかります．

さて，真のシステム応答の形式がわかったので，後は最悪ケースの$\boldsymbol{\Delta}$を考えれば良さそうです．特に，
* $\left\|\mathcal{D}_{\mathcal{A}}\right\|_{2 \rightarrow 2} \leq \epsilon_A$
* $\left\|\mathcal{D}_{\mathcal{B}}\right\|_{2 \rightarrow 2} \leq \epsilon_B$

である状況を考えましょう．また，簡単のために$\Sigma_w=I$とします．

このとき，次を解けばロバスト制御ができそうです．

$$
\begin{aligned}
\min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u}\max_{\left\|\mathcal{D}_{\mathcal{A}}\right\| \leq \epsilon_A,\left\|\mathcal{D}_{\mathcal{B}}\right\| \leq \epsilon_B} & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right](I+\boldsymbol{\Delta})^{-1}\right\|_F^2 \\
\text { subject to } & {[I-Z \widehat{\mathcal{A}}-Z \hat{\mathcal{B}}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]=I . }
\end{aligned}
$$

一方で，これはもう凸最適化問題ではありません．
解きやすくするために，次の変形を施します．

$$
\begin{aligned}
\left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\mathbf{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right](I+\boldsymbol{\Delta})^{-1}\right\|_F^2 & \leq\left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\right\|_F^2\left\|(I+\boldsymbol{\Delta})^{-1}\right\|_{2 \rightarrow 2}^2 \\
& \leq\left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]\right\|_F^2\left(\sum_{t=0}^{T-1}\|\boldsymbol{\Delta}\|_{2 \rightarrow 2}^t\right)^2,
\end{aligned}
$$

最初の不等式はFrobeniusノルムとスペクトルノルムの[submultiplicative property](https://en.wiktionary.org/wiki/submultiplicative)，２つ目の不等式は$\boldsymbol{\Delta}$がブロック下三角行列であることを使っています．

さらに，$\boldsymbol{\Delta}$のスペクトルノルムは

$$
\|\boldsymbol{\Delta}\|_{2 \rightarrow 2}=\left\|Z\left[\begin{array}{ll}
\mathcal{D}_{\mathcal{A}} & \mathcal{D}_{\mathcal{B}}
\end{array}\right]\left[\begin{array}{l}
\hat{\mathbf{\Phi}}_x \\
\hat{\boldsymbol{\Phi}}_u
\end{array}\right]\right\|_{2 \rightarrow 2} \leq \sqrt{2}\left\|\begin{array}{l}
\epsilon_A \boldsymbol{\Phi}_x \\
\epsilon_B \boldsymbol{\Phi}_u
\end{array}\right\|_{2 \rightarrow 2} .
$$

でバウンドできます（TODO: 証明．）


以上から，ロバストLQRは次の凸最適化問題で近似できます：

$$
\begin{aligned}
\min _\tau\left(\sum_{t=0}^{T-1} \tau^t\right)^2 \min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u} & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\right\|_F^2 \\
\text { subject to } & {[I-Z \widehat{\mathcal{A}}-Z \hat{\mathcal{B}}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]=I } \\
& \sqrt{2}\left\|\begin{array}{c}
\epsilon_A \boldsymbol{\Phi}_x \\
\epsilon_B \boldsymbol{\Phi}_u
\end{array}\right\|_{2 \rightarrow 2} \leq \tau .
\end{aligned}
$$

ちなみにこの最適化を解くことで得られる制御器が「真の性能でどれだけ性能が落ちるか？」は解析することができますが，かなりめんどくさいぽいです（[System Level Synthesis](https://arxiv.org/abs/1904.01634)の式(2.30)付近）．

上で見たスペクトルノルムの変形を考えると，
$A(\boldsymbol{\Phi})=
\left[
\begin{array}{c}
\epsilon_A \boldsymbol{\Phi}_x \\
\epsilon_B \boldsymbol{\Phi}_u
\end{array}
\right]
$
として，

$$
\left\|A(\boldsymbol{\Phi}) \right\|_{2 \rightarrow 2} \leq \tau / \sqrt{2}
$$
なので，$\|A\|_2 \leq s$ if and only if $A^T A \preceq s^2 I$ (and $\left.s \geq 0\right)$を使えば，

$$
A^T(\boldsymbol{\Phi})A(\boldsymbol{\Phi}) \preceq \frac{\tau^2}{2} I
$$

と同じです．
さらに$A^T A \preceq t^2 I($ and $t \geq 0) \Longleftrightarrow\left[\begin{array}{cc}t I & A \\ A^T & t I\end{array}\right] \succeq 0$を使って変形すれば，

$$
\left[\begin{array}{cc}\frac{\tau}{\sqrt{2}} I & A \\ A^T & \frac{\tau}{\sqrt{2}} I\end{array}\right] \succeq 0
$$

とできます．

**注記**：この$\tau$の最適化はめんどくさいっぽい？
[Adaptive Control of the Linear Quadratic Regulator](https://github.com/modestyachts/robust-adaptive-lqr/tree/master)の辺を見ると$\tau$は定数で扱ってるみたい（もしくは後で線形探索する）

In [25]:
import cvxpy as cp


def solve_robust_LQR(Ahat, Bhat, Q, R, T, x0, eps_A, eps_B, tau=0.9):
    assert_ABQR_consistent(Ahat, Bhat, Q, R)
    assert T >= 1
    n, p = Bhat.shape
    Phi = cp.Variable(((T+1)*(n+p), (T+1)*n))
    Phi_x = Phi[:(T+1)*n]
    Phi_u = Phi[(T+1)*n:]
 
    Z = make_down_shift_Z(T, n)
    Q_sqrt = psd_sqrt(Q)
    R_sqrt = psd_sqrt(R)
    A_block, B_block = make_block_AB(T, Ahat, Bhat)
    Q_block, R_block = make_block_QR(T, Q_sqrt, R_sqrt)
    QR_block = block_diag(Q_block, R_block)

    # コストを定義します
    # tau = cp.Variable()
    w = np.zeros((T+1)*n)
    w[:n] = x0
    Tau = sum([tau ** i for i in range(T)]) ** 2
    cost = cp.norm(QR_block @ Phi @ w, p="fro") * Tau

    # Phi用の制約を定義します
    I = np.eye(Z.shape[0])
    const_block = np.block([I - Z @ A_block, - Z @ B_block])

    # ロバスト用の制約を定義します
    A_Phi = cp.bmat([[eps_A * Phi_x], [eps_B * Phi_u]])
    tI1 = np.eye(A_Phi.shape[0]) * tau / np.sqrt(2)
    tI2 = np.eye(A_Phi.shape[1]) * tau / np.sqrt(2)
    robust_const_mat = cp.bmat([[tI1, A_Phi], [A_Phi.T, tI2]])

    # const = [const_block @ Phi == I, robust_const_mat >> 0, tau >= 0]
    const = [const_block @ Phi == I, robust_const_mat >> 0]

    prob = cp.Problem(cp.Minimize(cost), const)
    prob.solve()

    if prob.status == cp.OPTIMAL:
        Phi = Phi.value
        Phi_x = Phi[:(T+1)*n]
        Phi_u = Phi[(T+1)*n:]
        K = Phi_u @ np.linalg.inv(Phi_x)
        return True, K, prob.value
    else:
        return False, None, None

In [26]:
def sample_2_to_2_ball(A, eps):
    """行列Aからeps以内の行列をサンプルします"""
    Asample = np.array(A)
    Delta_A = np.random.normal(size=Asample.shape)
    Delta_A = Delta_A / np.linalg.norm(Delta_A, ord=2) * eps
    Asample += Delta_A
    TOL = 1e-7
    assert (np.linalg.norm(Asample - A, ord=2) <= eps + TOL), str(np.linalg.norm(Asample - A, ord=2))
    return Asample


eps_A, eps_B = 0.3, 0.3
Ahat = sample_2_to_2_ball(A, eps_A)
Bhat = sample_2_to_2_ball(B, eps_B)

x0 = np.array([0.1, -0.2, 0.1, -0.1])

_, K, cost = solve_LQR_no_noise(Ahat, Bhat, Q, R, T, x0)
_, K_Hinfty, cost = solve_LQR_H_infty(Ahat, Bhat, Q, R, T)
_, robust_K, cost = solve_robust_LQR(Ahat, Bhat, Q, R, T, x0, eps_A, eps_B)
states, inputs = sls_roll_forward(A, B, K, x0, T, 0.1)
H_states, H_inputs = sls_roll_forward(A, B, K_Hinfty, x0, T, 0.1)
robust_states, robust_inputs = sls_roll_forward(A, B, robust_K, x0, T, 0.1)

print("LQRのコスト", cost)
print("通常のLQRがRolloutしたコスト", cost_LQR_samples(states, inputs, Q, R))
print("H∞LQRがRolloutしたコスト", cost_LQR_samples(H_states, H_inputs, Q, R))
print("Robust LQRがRolloutしたコスト", cost_LQR_samples(robust_states, robust_inputs, Q, R))

LQRのコスト 6.779850970695929
通常のLQRがRolloutしたコスト 0.5828623676657916
H∞LQRがRolloutしたコスト 0.4414484968794523
Robust LQRがRolloutしたコスト 0.7995027552634487


In [80]:
fig = plt.figure(figsize=(5*3, 4))
camera = Camera(fig)
draw_inverted_pendulum(camera, [states, H_states, robust_states], ["LQR", "H∞ LQR", "robust LQR"])
animation = camera.animate()
animation.save("./figs/inv_pend_robust_sls.gif")
plt.close()

![sls](./figs/inv_pend_robust_sls.gif)

### ノイズありの制約付きLQR

参考
* [Safely Learning to Control the Constrained Linear Quadratic Regulator](https://arxiv.org/abs/1809.10121)

次の制約付きLQR問題を考えてみます．（これは$\mathcal{H}_\infty$ではありません！）

$$
\begin{array}{rl}
\min _{x_t, u_t} & \sum_{t=0}^T x_t^{\top} Q_t x_t+u_t^{\top} R_t u_t \\
\text { subject to } & x_{t+1}=A x_t+B u_t, t=0, \ldots, T-1 \\
& x_0 \text { random; } \quad F_x x_t \leq b_x, \quad F_u u_t \leq b_u \\
& \forall t, \forall\left\{w_t:\left\|w_t\right\|_{\infty} \leq \sigma_w\right\}
\end{array}
$$

これはベクトル形式で表せば，次と等価です．

$$
\begin{aligned}
\min _{\mathbf{x}, \mathbf{u}} & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\mathbf{x} \\
\mathbf{u}
\end{array}\right]\right\|_F^2 \\
\text { subject to } & \mathbf{x}=Z \mathcal{A} \mathbf{x}+Z \mathcal{B} \mathbf{u}+\mathbf{w} \\
& F_x\mathbf{x} \leq b_x, F_u\mathbf{u} \leq b_u, 
\end{aligned}
$$

であり，更にSLSを使うと

$$
\begin{aligned}
\min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u}  & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \right\|_F^2 \\
\text { subject to } & {[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I}\\
&F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\mathbf{w} \leq b
\end{aligned}
$$

として書けます．ここで，[ロバスト最適化](OPT_robust_optimization.ipynb)でやったように，$\|\cdot\|_\infty$の最悪ケースのノイズを制約に付加してみましょう．

つまり，

$$
\max_{\|\mathbf{w}\|_\infty \leq \sigma_w}F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\mathbf{w} \leq b
$$

を考えます．
この左辺は
$$
\max_{\|\mathbf{w}\|_\infty \leq \sigma_w}F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\mathbf{w}
=
\sigma_w
\left\|
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]
\right\|_{\infty\to \infty}
$$

と同じです．
よってまとめると，

$$
\begin{aligned}
\min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u}  & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \right\|_F^2 \\
\text { subject to } & {[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I}\\
&\sigma_w
\left\|
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]
\right\|_{\infty\to \infty} \leq b
\end{aligned}
$$

を解けば良いことがわかります．


### ノイズありの制約付きロバストLQR

上では真の行列$A, B$がわかっている場合について扱いました．
今回は$\hat{A}, \hat{B}$しかわからない状況を考えます．

上で見たように，ロバストLQRは次を扱います．

$$
\begin{aligned}
\min _\tau\left(\sum_{t=0}^{T-1} \tau^t\right)^2 \min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u} & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\right\|_F^2 \\
\text { subject to } & {[I-Z \widehat{\mathcal{A}}-Z \hat{\mathcal{B}}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]=I } \\
& \sqrt{2}\left\|\begin{array}{c}
\epsilon_A \boldsymbol{\Phi}_x \\
\epsilon_B \boldsymbol{\Phi}_u
\end{array}\right\|_{2 \rightarrow 2} \leq \tau .
\end{aligned}
$$

これに制約を追加していくわけですが，今回は$A, B$の最悪ケースについて制約を保証するので，単純に制約を追加するだけでは不十分です．

真のシステムでの制約を満たすには，
$$
\begin{aligned}
b &\geq 
\max_{\|\mathbf{w}\|_\infty \leq \sigma_w}
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right](I + \mathbf{\Delta})^{-1}\mathbf{w}\\
&=
\max_{\|\mathbf{w}\|_\infty \leq \sigma_w}
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\left(I - \mathbf{\Delta}(I + \mathbf{\Delta})^{-1}\right)\mathbf{w}\\
&=\max_{\|\mathbf{w}\|_\infty \leq \sigma_w}
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\mathbf{w} - 
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]
\mathbf{\Delta}(I + \mathbf{\Delta})^{-1}\mathbf{w}\\
\end{aligned}
$$
を達成する必要があります．
最初の項はロバストなしの場合と同じですね．

２つ目の項について考えてみましょう．

$$
\begin{aligned}
\|
\mathbf{\Delta}(I + \mathbf{\Delta})^{-1}\mathbf{w}
\|_{\infty\to \infty}
&\leq 
\|
\mathbf{\Delta}(I + \mathbf{\Delta})^{-1}\|_{\infty\to \infty}\|\mathbf{w}
\|_\infty\\
&\leq 
\frac{\|\mathbf{\Delta}\|_{\infty\to \infty}}{1 - \|\mathbf{\Delta}\|_{\infty \to \infty}}
\|\mathbf{w}
\|_\infty\\
\end{aligned}
$$

後は$\|\mathbf{\Delta}\|_{\infty \to \infty}$を上から押さえれば良さそうです．
上で見たロバストLQRと同様にすれば，

$$
\|\mathbf{\Delta}\|_{\infty \to \infty} \leq 
\sqrt{2}\left\|\begin{array}{l}
\epsilon_{A, \infty} \boldsymbol{\Phi}_x \\
\epsilon_{B, \infty} \boldsymbol{\Phi}_u
\end{array}\right\|_{\infty \to \infty}
$$

でバウンドできることがわかります．（TODO: 証明）
全部合体して，


$$
\begin{aligned}
\min _{\tau, \gamma}\left(\sum_{t=0}^{T-1} \tau^t\right)^2 \min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u} & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\right\|_F^2 \\
\text { subject to } & {[I-Z \widehat{\mathcal{A}}-Z \hat{\mathcal{B}}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]=I } \\
& \sqrt{2}\left\|\begin{array}{c}
\epsilon_A \boldsymbol{\Phi}_x \\
\epsilon_B \boldsymbol{\Phi}_u
\end{array}\right\|_{2 \rightarrow 2} \leq \tau \quad
\left\|\begin{array}{c}
\epsilon_{A, \infty} \boldsymbol{\Phi}_x \\
\epsilon_{B, \infty} \boldsymbol{\Phi}_u
\end{array}\right\|_{\infty \rightarrow \infty} \leq \gamma \\
&
\frac{\sigma_w}{1 - \gamma}
\left\|
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]
\right\|_{\infty\to \infty} 
\leq b
\end{aligned}
$$

**コメント** 論文では制約上で∞ノルムを考えているが，$2\to 2$のノルムなら一個省略できるかも．つまり，

$$
\begin{aligned}
\min _{\tau, \gamma}\left(\sum_{t=0}^{T-1} \tau^t\right)^2 \min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u} & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\right\|_F^2 \\
\text { subject to } & {[I-Z \widehat{\mathcal{A}}-Z \hat{\mathcal{B}}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]=I } \\
& \sqrt{2}\left\|\begin{array}{c}
\epsilon_A \boldsymbol{\Phi}_x \\
\epsilon_B \boldsymbol{\Phi}_u
\end{array}\right\|_{2 \rightarrow 2} \leq \tau\;, \quad
\frac{\sigma_w}{1 - \tau}
\left\|
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]
\right\|_{2\to 2} 
\leq b
\end{aligned}
$$

でもいけるかも？
制約付きLQR（ノイズなし）は

$$
\begin{aligned}
\min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u}  & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right] \mathbf{w} \right\|_F^2 \\
\text { subject to } & {[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I}\\
&
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\mathbf{w}
\leq b
\end{aligned}
$$

制約付きLQR（$\mathcal{H}_\infty$）は

$$
\begin{aligned}
\min _{\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u}  & \left\|\left[\begin{array}{cc}
\mathcal{Q}^{\frac{1}{2}} & 0 \\
0 & \mathcal{R}^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\right\|_F^2 \\
\text { subject to } & {[I-Z \mathcal{A}-Z \mathcal{B}]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I}\\
&\sigma_w
\left\|
F \left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]
\right\|_{2\to 2} \leq b
\end{aligned}
$$



かな？


In [27]:
def solve_const_LQR_no_noise(A, B, Q, R, T, x0, F: np.ndarray, b: np.ndarray):
    """制約付きSLSを解きます．"""

    assert_ABQR_consistent(A, B, Q, R)
    assert T >= 1
    n, p = B.shape

    # コストを定義します
    Q_sqrt = psd_sqrt(Q)
    R_sqrt = psd_sqrt(R)

    Z = make_down_shift_Z(T, n)
    A_block, B_block = make_block_AB(T, A, B)
    Q_block, R_block = make_block_QR(T, Q_sqrt, R_sqrt)
    QR_block = block_diag(Q_block, R_block)

    Phi = cp.Variable(((T+1)*(n+p), (T+1)*n))

    w = np.zeros((T+1)*n)
    w[:n] = x0

    cost = cp.norm(QR_block @ Phi @ w, p="fro")

    # SLSの制約を定義します
    I = np.eye(Z.shape[0])
    const_block = np.block([I - Z @ A_block, - Z @ B_block])
    const = [const_block @ Phi == I]

    # 制約付き制御用の制約を定義します
    const = [const_block @ Phi == I, F @ Phi @ w <= b]

    prob = cp.Problem(cp.Minimize(cost), const)
    prob.solve()

    if prob.status == cp.OPTIMAL:
        Phi = Phi.value
        Phi_x = Phi[:(T+1)*n]
        Phi_u = Phi[(T+1)*n:]
        K = Phi_u @ np.linalg.inv(Phi_x)
        return True, K, prob.value
    else:
        return False, None, None

In [69]:
n, p = B.shape
T = 30

# 今回は一定ステップ後に右に倒れては行けないという制約にしてみます
# そのため，各時刻毎に制約を課します
F = np.zeros((T+1, (T+1)*(n+p)))
b = np.zeros((T+1))

for t in range(5, int(T)):
    F[t, t * (n+p) + 2] = 1.0
    b[t] = - 0.05

_, K, cost = solve_LQR_no_noise(A, B, Q, R, T, x0)
states, inputs = sls_roll_forward(A, B, K, x0, T)

_, K, cost = solve_const_LQR_no_noise(A, B, Q, R, T, x0, F, b)
const_states, const_inputs = sls_roll_forward(A, B, K, x0, T)

print("LQRのコスト", cost)
print("通常のLQRがRolloutしたコスト", cost_LQR_samples(states, inputs, Q, R))
print("制約付きLQRがRolloutしたコスト", cost_LQR_samples(const_states, const_inputs, Q, R))

LQRのコスト 0.7235674949723006
通常のLQRがRolloutしたコスト 0.0011462474142719418
制約付きLQRがRolloutしたコスト 0.21302923314279662


In [70]:
fig = plt.figure(figsize=(5, 4))
camera = Camera(fig)
draw_inverted_pendulum(camera, [states, const_states], ["LQR", "Constrained LQR"])
animation = camera.animate()
animation.save("./figs/inv_pend_sls_const.gif")
plt.close()

![sls](./figs/inv_pend_sls_const.gif)